<p><img style="padding: 0 15px; float: left;" src="Images/FTLogo300.jpg" alt="FT Crusader Logo" Width='140' Height= '250'/></p> 
<p> <h2> Frogtown DataLoad Workbook, 05/06/18</h2> <a name="tc"></a>
<p> <h3> By Frogtown Crusader (Abu Nayeem)</h3>

This is just clean coded process going over the uploading steps assuming all the preliminary steps after decoder are made


## Table of contents <a name="content"></a>
* [Data Setup](#setup)
* [Create Variables](#create)
* [Intersection Table: Preparation](#intersection)
* [Address Table: Preparation](#address)
* [Preliminary Data Cleaning](#pre)


### Data Setup <a name="setup"></a>

In [3]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline 
from IPython.display import HTML
from IPython.display import display
import requests # library to handle requests

#Load Data
#df_crime = pd.read_csv('Datasets/Crime_Incident_Report_-_Dataset.csv')

#rename columns
#cols= ['Case','Date','Time','Code','IncType','Incident','Grid','NNum','Neighborhood','Block','CallDispCode','CallDisposition', 'Count']
#df_crime.columns= cols

#selection for Frogtown and nearby area
#df=df_crime.query('Grid in [66,67, 68, 86, 87,88,89, 90, 91, 92,106,107,108,109,110]')

#Socarata upload Method
from sodapy import Socrata

#New Upload Method Get Information from Socrata API
client = Socrata("information.stpaul.gov", None)

#Easier to bulk upload
results = client.get("gppb-g9cg", limit=1000000)
df = pd.DataFrame.from_records(results)
# Find Max Date Value
results = client.get("gppb-g9cg", limit=1)
r_max = pd.DataFrame.from_records(results)

#rename columns [Note the order of Columns have changed]
cols= ['Block','CallDispCode','CallDisposition','Case','Code', 'Count','Date','Grid','Incident','IncType','Neighborhood','NNum','Time']
df.columns= cols
df=df.dropna()
df = df.astype({"Case": int, "Code": int, "Grid":float, "NNum":int,"Count":int})
#select respective Grids of interest
df=df.query("Grid in (66,67, 68, 86, 87,88,89, 90, 91, 92,106,107,108,109,110)")

df.head(4)

,Block,CallDispCode,CallDisposition,Case,Code,Count,Date,Grid,Incident,IncType,Neighborhood,NNum,Time
3,55X CENTRAL AV W,A,Advised,20113203,9954,1,2020-06-10T23:20:42.000,109.0,Proactive Police Visit,Proactive Police Visit,8 - Summit/University,8,2020-06-10T23:20:42.000
4,11X EMPIRE DR,RR,Report Written,20113200,311,1,2020-06-10T23:18:00.000,91.0,Robbery,"Robbery, Highway, Firearm",7 - Thomas/Dale(Frogtown),7,2020-06-10T23:18:00.000
7,93X CENTRAL AV W,RR,Report Written,20113190,2619,1,2020-06-10T23:05:00.000,107.0,Discharge,"Weapons, Discharging a Firearm in the City Limits",8 - Summit/University,8,2020-06-10T23:05:00.000
14,76X UNIVERSITY AV W,A,Advised,20113148,2619,1,2020-06-10T21:53:57.000,88.0,Discharge,"Weapons, Discharging a Firearm in the City Limits",7 - Thomas/Dale(Frogtown),7,2020-06-10T21:53:57.000


### Create New Variables <a name="create"></a>

In [5]:
#Add Time Variables
df= df[df.Case != 18254093] #messed up time variable

#Convert Date to Datetime!
from datetime import datetime

df['DateTime']= pd.to_datetime(df['Date']) # Create new column called DateTime
df['Year']= df['DateTime'].dt.year #create year column
df['DayofWeek']=df['DateTime'].dt.dayofweek #create day of the week column where default 0=Monday
df['Weekend'] = df['DayofWeek'].apply(lambda x: 1 if (x>4)  else 0) #Create a weekend category
df['Month'] = df['DateTime'].dt.month # Create Month Category
df['Day'] = df['DateTime'].dt.day #Create Day of the Current month
df['DayYear'] = df['DateTime'].dt.dayofyear  #Create Day of the year (0-365)
df['Day_Max'] = df.iloc[0,-1] #selects uptodate day; NOTE: the data is sorted chronologically

#Hour Data
df['TimeHour']= pd.to_datetime(df['Time'])
df['Hour'] = df['TimeHour'].dt.hour.astype(int) #Create Hour Colum
df['LateNight'] = df['Hour'].apply(lambda x: 1 if (x>21 or x<5)  else 0) #Latenight designation from 10Pm to 6PM

#Creating the intersection Column. Note: the Block column has the address information
df.Block = df.Block.astype(str) #first change the type to string
df['Block']= df['Block'].str.lower() #lowercase string to create uniformity

#While scanning the data I noticed that all intersections had "&" 
df['Intersection'] = df['Block'].apply(lambda x: 1 if '&' in x else 0) #intersection

df.head(5)

,Block,CallDispCode,CallDisposition,Case,Code,Count,Date,Grid,Incident,IncType,...,DayofWeek,Weekend,Month,Day,DayYear,Day_Max,TimeHour,Hour,LateNight,Intersection
3,55x central av w,A,Advised,20113203,9954,1,2020-06-10T23:20:42.000,109.0,Proactive Police Visit,Proactive Police Visit,...,2,0,6,10,162,0,2020-06-10 23:20:42,23,1,0
4,11x empire dr,RR,Report Written,20113200,311,1,2020-06-10T23:18:00.000,91.0,Robbery,"Robbery, Highway, Firearm",...,2,0,6,10,162,0,2020-06-10 23:18:00,23,1,0
7,93x central av w,RR,Report Written,20113190,2619,1,2020-06-10T23:05:00.000,107.0,Discharge,"Weapons, Discharging a Firearm in the City Limits",...,2,0,6,10,162,0,2020-06-10 23:05:00,23,1,0
14,76x university av w,A,Advised,20113148,2619,1,2020-06-10T21:53:57.000,88.0,Discharge,"Weapons, Discharging a Firearm in the City Limits",...,2,0,6,10,162,0,2020-06-10 21:53:57,21,0,0
15,71x vanburen av,R,Report Written,20113144,1400,1,2020-06-10T21:42:53.000,88.0,Vandalism,Criminal Damage to Property,...,2,0,6,10,162,0,2020-06-10 21:42:53,21,0,0


### Prepare Intersection table <a name="intersection"></a>

In [6]:
#Load clean intersections key
df_Intersection_key = pd.read_csv('Datasets/Intersection_key_clean.csv')

# Create a new dateframe specifying only intersections from primary dataset
dfI=df.query('Intersection ==1')
print('The intersection table dimension are ' + str(dfI.shape))
#print(dfI.Block.head(10))


#Split the strings
new=dfI['Block'].str.split("& ", n = 1, expand = True) 
dfI['Inter2']= new[1]
new=dfI['Block'].str.split(" ", n = 1, expand = True) #Note the code specifies the first time a space occured
dfI['Inter1']=new[0]

#Create the IndexKey; recall we prepared the IntersectionKey where it considers any order
dfI['IndexKey']= dfI['Inter1']+ '_' + dfI['Inter2']
dfI.reset_index()
dfI=pd.merge(dfI, df_Intersection_key, on='IndexKey', how='left')



The intersection table dimension are (7593, 25)


In [7]:
#Drop rows with missing coordinates
dfI=dfI[dfI['Coordinates'].notnull()]

# Separate Latitude and Longitude 
new=dfI['Coordinates'].str.split(",", n = 1, expand = True) 
# making seperate first name column from new data frame 
dfI['Latitude']= pd.to_numeric(new[0]) #pd.to_numeric convert it to float
dfI['Longitude']= pd.to_numeric(new[1])

#Renaming columns

dfI['Block']=dfI['OutputKey'] #for practical purposes it makes sense
Drop_col=['Inter2','Inter1', 'IndexKey', 'Coordinates', 'OutputKey']
dfI_Final=dfI.drop(Drop_col, axis=1,)
dfI_Final.head(5)


,Block,CallDispCode,CallDisposition,Case,Code,Count,Date,Grid,Incident,IncType,...,Month,Day,DayYear,Day_Max,TimeHour,Hour,LateNight,Intersection,Latitude,Longitude
0,arundel_aurora,A,Advised,20113091,9954,1,2020-06-10T20:19:41.000,109.0,Proactive Police Visit,Proactive Police Visit,...,6,10,162,0,2020-06-10 20:19:41,20,0,1,44.954895,-93.118649
1,carroll_grotto,A,Advised,20113086,9954,1,2020-06-10T20:12:32.000,108.0,Proactive Police Visit,Proactive Police Visit,...,6,10,162,0,2020-06-10 20:12:32,20,0,1,44.950342,-93.131401
2,arundel_charles,A,Advised,20112999,9954,1,2020-06-10T17:21:08.000,89.0,Proactive Police Visit,Proactive Police Visit,...,6,10,162,0,2020-06-10 17:21:08,17,0,1,44.957576,-93.118668
3,arundel_university,A,Advised,20112989,9954,1,2020-06-10T17:11:41.000,89.0,Proactive Police Visit,Proactive Police Visit,...,6,10,162,0,2020-06-10 17:11:41,17,0,1,44.955853,-93.118679
6,grotto_thomas,A,Advised,20112815,9954,1,2020-06-10T12:42:58.000,88.0,Proactive Police Visit,Proactive Police Visit,...,6,10,162,0,2020-06-10 12:42:58,12,0,1,44.959356,-93.131431


### Prepare Address Decoder Table <a name="address"></a>

It is so remarkably short

In [8]:
#Split Data
dfW=df.query('Intersection==0')

#Load Complete Decoder Key
df_C= pd.read_csv('Datasets/SemiKey.csv')
df_C= df_C[['Block','Latitude','Longitude']]

# Merge with the dataset and remove missing values
dC=pd.merge(dfW, df_C, on='Block', how='left')
dC=dC.fillna('Mi')
dC=dC.query('Latitude != "Mi"')

#Bringing the data back together 
fg= dfI.append(dC, ignore_index=True)

### Preliminary Pre-Cleaning steps <a name="clean"></a>

It is good to do the data pre-cleaning steps here to reduce clutter on a working notebook. A few edits include renaming some values, clustering crimes together and creating some dummies. It is saved in a csv, which is used for execution

In [9]:
#Few Quick Edits
fg.CallDisposition.loc[(fg['CallDisposition'] == 'G - Gone on Arrival')] = 'Gone on Arrival'
fg.CallDisposition.loc[(fg['CallDisposition'] == 'A - Advised')] = 'Advised'
fg.CallDisposition.loc[(fg['CallDisposition'] == 'RR - Report Written')] = 'Report Written'
fg.Incident.loc[(fg['Incident'] == 'Simple Asasult Dom.')] = 'Simple Assault Dom.'
fg.Incident.loc[(fg['Incident'] == 'Graffiti')] = 'Vandalism'
fg.Incident.loc[fg["Incident"].isin([ "Rape","Agg. Assault",'Homicide'])]= 'Violent'
fg.Incident.loc[fg["Incident"].isin(["Simple Assault Dom.","Agg. Assault Dom."])]= 'Domestic Assault'

#[fg["Incident"].isin(["Simple Assault Dom.", "Rape"])


#Create a dummy for each crime category
fg= pd.concat([fg,pd.get_dummies(fg['Incident'])], axis=1)
fg= pd.concat([fg,pd.get_dummies(fg['CallDisposition'])], axis=1)

fg.to_csv('Datasets/FGCrime_Final.csv', encoding='utf-8', index=False)

In [7]:
fg.query('Year==2020')

,Block,CallDispCode,CallDisposition,Case,Code,Coordinates,Count,Date,DateTime,Day,...,Vandalism,Violent,10 - Como,11 - Hamline/Midway,13 - Union Park,7 - Thomas/Dale(Frogtown),8 - Summit/University,Advised,Gone on Arrival,Report Written
0,milton_thomas,A,Advised,20005493,9954,"44.959361, -93.139031",1,2020-01-08T20:42:50.000,2020-01-08 20:42:50,8,...,0,0,0,0,0,0,0,1,0,0
1,linden_mtairy,A,Advised,20005175,9954,"44.959025, -93.095111",1,2020-01-08T12:10:36.000,2020-01-08 12:10:36,8,...,0,0,0,0,0,0,0,1,0,0
2,kent_vanburen,RR,Report Written,20005107,2619,"44.962048, -93.123821",1,2020-01-08T10:02:00.000,2020-01-08 10:02:00,8,...,0,0,0,0,0,0,0,0,0,1
3,linden_arch,A,Advised,20005095,9954,"44.960767, -93.095135",1,2020-01-08T09:47:17.000,2020-01-08 09:47:17,8,...,0,0,0,0,0,0,0,1,0,0
4,arundel_aurora,A,Advised,20004865,9954,"44.954895, -93.118649",1,2020-01-07T22:08:02.000,2020-01-07 22:08:02,7,...,0,0,0,0,0,0,0,1,0,0
5,arundel_university,RR,Report Written,20004861,410,"44.955853, -93.118679",1,2020-01-07T21:15:00.000,2020-01-07 21:15:00,7,...,0,1,0,0,0,0,0,0,0,1
6,milton_thomas,A,Advised,20004151,9954,"44.959361, -93.139031",1,2020-01-06T23:29:05.000,2020-01-06 23:29:05,6,...,0,0,0,0,0,0,0,1,0,0
7,grotto_piercebutler,A,Advised,20004143,2619,"44.964722, -93.131433",1,2020-01-06T23:13:00.000,2020-01-06 23:13:00,6,...,0,0,0,0,0,0,0,1,0,0
8,marshall_western,A,Advised,20004134,9954,"44.948211, -93.116157",1,2020-01-06T22:50:39.000,2020-01-06 22:50:39,6,...,0,0,0,0,0,0,0,1,0,0
9,arundel_university,A,Advised,20003995,9954,"44.955853, -93.118679",1,2020-01-06T17:46:00.000,2020-01-06 17:46:00,6,...,0,0,0,0,0,0,0,1,0,0


In [ ]:
Ward1.query('ElectionDescription in ("11/08/2016 - STATE GENERAL", "08/09/2016 - STATE PRIMARY","11/07/2017 - MUNICIPAL GENERAL","11/07/2017 - SCHOOL DISTRICT GENERAL"  ,"11/08/2016 - STATE GENERAL","08/09/2016 - STATE PRIMARY","11/03/2015 - MUNICIPAL GENERAL","11/03/2015 - SCHOOL DISTRICT GENERAL","11/08/2011 - MUNICIPAL GENERAL","11/06/2018 - STATE GENERAL","08/14/2018 - STATE PRIMARY")' 